In [9]:
import tensorflow as tf
from tensorflow import keras
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import time

In [23]:
mnist =tf.keras.datasets.mnist
(train_images,_), (_,_) = mnist.load_data()
train_images=train_images.reshape(train_images.shape[0],28, 28, 1).astype('float32')

In [31]:
from tensorflow.data.Dataset import from_tensor_slices
train_images=(train_images-127.5)/127.5
BUFFER_SIZE=60000
BATCH_SIZE=256
train_dataset=from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

47040000

## GENERADOR DE RUIDO

In [37]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import (Dense, 
                                    Reshape, 
                                    Conv2DTranspose, 
                                    BatchNormalization,
                                    LeakyReLU)

def make_generator_model():
    model = Sequential()
    model.add(Dense(7*7*256, use_bias=False, input_shape=(100,)))

    model.add(Reshape((7,7,256)))

    model.add(Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))

    model.add(Conv2DTranspose(64, (5, 5), strides=(1, 1), padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))
    model.add(Conv2DTranspose(1, (5,5), strides=(2, 2), padding='same', activation='tanh'))

    return model


model = make_generator_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 12544)             1254400   
                                                                 
 reshape_1 (Reshape)         (None, 7, 7, 256)         0         
                                                                 
 conv2d_transpose_3 (Conv2D  (None, 14, 14, 128)       819328    
 Transpose)                                                      
                                                                 
 batch_normalization_2 (Bat  (None, 14, 14, 128)       512       
 chNormalization)                                                
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 14, 14, 128)       0         
                                                                 
 conv2d_transpose_4 (Conv2D  (None, 14, 14, 64)       